In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

In [ ]:
ckpt_dir = "/home/ccl/Code/svox2/opt/ckpt_auto/scannerf_light_short"
plot_dir = "/home/ccl/Downloads/plots"


def load_results(results_dir):
    c2ws = np.load(os.path.join(results_dir, "c2ws.npy"))
    psnr = np.load(os.path.join(results_dir, "psnr.npy"))
    ssim = np.load(os.path.join(results_dir, "ssim.npy"))
    lpips = np.load(os.path.join(results_dir, "lpips.npy"))
    return c2ws, psnr, ssim, lpips
    
splits = ["all_100", "all"]
# splits = ["all_100", "all_250", "all_500", "all"]
scenes = ["airplane1", "brontosaurus", "lego"]
# scenes = ["airplane1", "airplane2", "brontosaurus", "bulldozer1", "bulldozer2", "cheetah", "dump_truck1", "dump_truck2", "elephant", "excavator", "forklift", "giraffe", "helicopter1", "helicopter2", "lego", "lion", "plant1", "plant2", "plant3", "plant4", "plant5", "plant6", "plant7", "plant8", "plant9", "roadroller", "shark", "spinosaurus", "stegosaurus", "tiger", "tractor", "trex", "triceratops", "truck", "zebra"]
c2ws_test = {}
c2ws_train = {}
psnr_test = {}
psnr_train = {}


for split in splits:
    c2ws_test[split] = {}
    c2ws_train[split] = {}
    psnr_test[split] = {}
    psnr_train[split] = {}
    for scene in scenes:
        test_results_dir = os.path.join(ckpt_dir, split, scene, "test_renders")
        train_results_dir = os.path.join(ckpt_dir, split, scene, "train_renders")

        c2ws_test_i, psnr_test_i, ssim_test_i, lpips_test_i = load_results(test_results_dir)
        c2ws_train_i, psnr_train_i, ssim_train_i, lpips_train_i = load_results(train_results_dir)
        
        # cos_dist = np.matmul(directions, directions_train.transpose())
        # closest_train_index = np.argmax(cos_dist, axis=1)
        # cos_dist = np.max(cos_dist, axis=1)
        # rel_angles = np.arccos(cos_dist)

        c2ws_test[split][scene] = c2ws_test_i
        c2ws_train[split][scene] = c2ws_train_i
        psnr_test[split][scene] = psnr_test_i
        psnr_train[split][scene] = psnr_train_i

c2ws = next(iter(next(iter(c2ws_test.values())).values()))
center_pt = np.zeros(3)
directions = (c2ws[:,:3,3] - center_pt)
directions = directions / np.linalg.norm(directions, keepdims=True, axis=1)

angles_equator = np.arctan2(directions[:,0], directions[:,2])
angles_vert_x = np.arctan2(-directions[:,1], directions[:,0])
angles_vert_z = np.arctan2(-directions[:,1], directions[:,2])

# best_psnr = np.max(psnrs, axis=0)
# best_psnr_idx = np.argmax(psnrs, axis=0)

In [ ]:
def xyz(c2ws):
    return c2ws[:,0,3], c2ws[:,2,3], -c2ws[:,1,3]

def plot_filter(condition, labels, filename, angles):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    inlier = np.where(condition)
    outlier = np.where(np.logical_not(condition))
    ax.scatter(*xyz(c2ws[outlier]), label=labels[1])
    ax.scatter(*xyz(c2ws[inlier]), label=labels[0])
    ax.legend()
    fig.savefig(os.path.join(plot_dir, filename))
    
    return inlier[0]

equator = plot_filter(c2ws[:,1,3] > -0.25, ["equator", "not equator"], "equator_selection.jpeg", angles_equator)
vert_x = plot_filter(np.abs(c2ws[:,2,3]) < 0.1, ["vert_x", "not vert_x"], "vert_x_selection.jpeg", angles_vert_x)
vert_z = plot_filter(np.abs(c2ws[:,0,3]) < 0.1, ["vert_z", "not vert_z"], "vert_z_selection.jpeg", angles_vert_z)


In [ ]:
#%matplotlib inline

In [ ]:
def plot_quality(scene, theta, quality, img_ids, filename):
    fig = plt.figure()
    ax = fig.add_subplot(projection='polar')
    plt.ylabel("PSNR")
    plt.xlabel("absolute angle around scene origin")
    plt.title(f"Scene: {scene} - PSNR along vertical slice")
    for split in splits:
        r = quality[split][scene]
        ax.scatter(theta[img_ids], r[img_ids], label=f"{split}")

    ax.legend()
    fig.savefig(os.path.join(plot_dir, filename))
    return fig

fig = plot_quality("airplane1", angles_equator, psnr_test, equator, f"{scene}_psnr_equator.jpeg")
# plt.close(fig)
# fig = plot_quality(scene, angles_vert_x, psnr_test, vert_x, f"{scene}_psnr_vert_x.jpeg")
# plt.close(fig)
# fig = plot_quality(scene, angles_vert_z, psnr_test, vert_z, f"{scene}_psnr_vert_z.jpeg")
# plt.close(fig)



In [ ]:
def vis_sample(scene, img_id, desc, img_ids, angles, selected_split):
    fig = plt.figure(figsize=(10,12))
    ax1 = plt.subplot(311, projection='polar')
    ax2 = plt.subplot(312)
    ax3 = plt.subplot(313, projection='3d')
    theta = angles[img_ids]
    
    psnr = psnr_test[selected_split][scene][img_id]
    
    ax1.title.set_text(f"{desc} - img_id {img_id}")
    for j, split in enumerate(splits):
        r = psnr_test[split][scene]
        ax1.scatter(theta, r[img_ids], s=5, label=split)
    ax1.scatter(angles[img_id], psnr, s=50, label=f"current")
    ax1.legend()
    
    
    img_path = os.path.join(ckpt_dir, selected_split, scene, "test_renders", f'{img_id:04d}.png')
    img = np.asarray(Image.open(img_path))
    ax2.title.set_text(f"{desc} - PSNR: {psnr:.2f}")
    ax2.imshow(img)

    ax3.scatter(*xyz(c2ws_train[selected_split][scene]), s=5, label=f"train_{selected_split}")
    plot = ax3.scatter(*xyz(c2ws), c=psnr_test[selected_split][scene], cmap='hot', s=5, label=f"test_all")
    ax3.scatter(*xyz(c2ws[img_id:img_id+1]), s=5, label=f"current")
    ax3.legend()
    fig.colorbar(plot).set_label('PSNR')
    
    fig.savefig(os.path.join(plot_dir, f"{scene}_{desc}_{psnr:.01f}_{img_id}.png"))
    
vis_sample("airplane1", vert_z[0], "failure_case", vert_z, angles_vert_z, "all_100")

In [ ]:
sample_num = 2
scene = "lego"
selected_split = "all_100"

# filter_name = "equator"
# img_ids = equator
# angles = angles_equator

filter_name = "vert_x"
img_ids = vert_x
angles = angles_vert_x

# filter_name = "vert_z"
# img_ids = vert_z
# angles = angles_vert_z

fail_idx = np.argsort(psnr_test[selected_split][scene][img_ids])[:sample_num]
success_idx = np.argsort(psnr_test[selected_split][scene][img_ids])[-sample_num:][::-1]
print()
fig = plot_quality(scene, angles, psnr_test, img_ids, f"{scene}_psnr_{filter_name}.jpeg")
for img_id in list(img_ids[fail_idx]):
    vis_sample(scene, img_id, "failure_case", img_ids, angles, selected_split)
for img_id in list(img_ids[success_idx]):
    vis_sample(scene, img_id, "success_case", img_ids, angles, selected_split)

In [ ]:

fig = plt.figure(figsize=(10,8))
plt.title("locations of different training/validation samples")
ax = fig.add_subplot(projection='3d')

ax.scatter(*xyz(c2ws), s=5, label="val_all")
for split in splits:
    # plt.title(f"train locations {split}")
    ax.scatter(*xyz(c2ws_train[split][scene]), s=5, label=f"train_{split}")

ax.legend()
fig.savefig(f"/home/ccl/Downloads/plots/{scene}_sample_locations.jpeg")